# Part 1: The Rail Environment


In this notebook, we will see how to create, interact with and render our first railway systems.

# In a nutshell

In [ ]:
# Install Flatland
!git clone https://gitlab.aicrowd.com/flatland/flatland.git/ --branch 223_UpdateEditor_55_notebooks
%cd flatland
!pip install -e .

In [ ]:
import numpy as np

from flatland.envs.rail_env import RailEnv
from flatland.envs.rail_generators import rail_from_manual_specifications_generator

In [ ]:
# Create a fixed railway from a 2D array of (tile type, rotation) tuples
specs = [[(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)],
         [(0, 0), (0, 0), (0, 0), (0, 0), (7, 0), (0, 0)],
         [(7, 270), (1, 90), (1, 90), (1, 90), (2, 90), (7, 90)],
         [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]]

rail_shape = np.array(specs).shape

fixed_env = RailEnv(width=rail_shape[1],
              height=rail_shape[0],
              rail_generator=rail_from_manual_specifications_generator(specs),
              number_of_agents=1
              )

# Call reset() to initialize the env
observation = fixed_env.reset()

In [ ]:
# We will study observations in details in the next notebook.

# As a first approach: by default, for each agent, the observation contains 3 arrays
agent_handle = 0
print('Agent {} observations:'.format(agent_handle))
agent_obs = observation[0][agent_handle]

print('Transition map')
print(agent_obs[0])
print()

print('State of all agents in the environment')
print(agent_obs[1])
print()

print('Position and targets of all the agents')
print(agent_obs[2])
print()

# Additional information from the observation
print('Additional information')
print(observation[1])

In [ ]:
from flatland.envs.rail_env import RailEnvActions

# Calling step() returns dictionaries
next_obs, all_rewards, done, _ = env.step({agent_handle: RailEnvActions.MOVE_FORWARD})

print('Rewards for each agent:')
print(all_rewards)

print('Done for each agent:')
print(done)

In [ ]:
import PIL
from flatland.utils.rendertools import RenderTool

# Render the environment
def render_env(env):
    env_renderer = RenderTool(env, gl="PILSVG")
    env_renderer.render_env()

    image = env_renderer.get_image()
    pil_image = PIL.Image.fromarray(image)
    display(pil_image)

render_env(fixed_env)

In [ ]:
from flatland.envs.rail_generators import random_rail_generator

# Random rail networks can be built from a list of probability per cell type
transition_probability = [1.0,  # empty cell - Case 0
                          1.0,  # Case 1 - straight
                          1.0,  # Case 2 - simple switch
                          0.3,  # Case 3 - diamond drossing
                          0.5,  # Case 4 - single slip
                          0.5,  # Case 5 - double slip
                          0.2,  # Case 6 - symmetrical
                          0.0,  # Case 7 - dead end
                          0.2,  # Case 8 - turn left
                          0.2,  # Case 9 - turn right
                          1.0]  # Case 10 - mirrored switch

random_env = RailEnv(width=15,
              height=15,
              rail_generator=random_rail_generator(
                        cell_type_relative_proportion=transition_probability
                        ),
              number_of_agents=1)

random_env.reset();

In [ ]:
render_env(random_env)

In [ ]:
# Let's "train" a random agent in this environment
class RandomAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size

    def act(self, state):
        return np.random.choice(np.arange(self.action_size))

    def step(self, memories):
        # This is where the agent would learn
        return

In [ ]:
agent = RandomAgent(218, 4)
obs, info = env.reset()

score = 0
action_dict = dict()

for step in range(500):
    render_env(env)
    
    for agent_handle in range(env.get_num_agents()):
        action = agent.act(obs[agent_handle])
        action_dict.update({a: action})
        print('Agent {} takes action {}'.format(agent_handle, RailEnvActions.to_char(action)))

    next_obs, all_rewards, done, _ = env.step(action_dict)
    
    
    for a in range(env.get_num_agents()):
        agent.step((obs[a], action_dict[a], all_rewards[a], next_obs[a], done[a]))
        score += all_rewards[a]
        
    print('Step {}, return = {}'.format(step, score))

    if done['__all__']:
        print('All the agents are done!')
        break
        
    print()

print('Episode done in {} steps, total return = {}'.format(step, score))
render_env(env)

**That's it!** You've created and rendered your first rail environments, and ran a random agent in it.

# In more details

Let's go again through the code above step by step.

Fixed rail network
---

There are multiple ways to generate a rail network. The simpler one is to describe it explicitely, as such:

In [ ]:
specs = [[(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)],
         [(0, 0), (0, 0), (0, 0), (0, 0), (7, 0), (0, 0)],
         [(7, 270), (1, 90), (1, 90), (1, 90), (2, 90), (7, 90)],
         [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]]

`specs` is a 2-dimensional array of tuples:

In [ ]:
import numpy as np

rail_shape = np.array(specs).shape
rail_shape

The `specs` array represent a 4 by 6 2D grid of tuples. In each tuple, the first element represent the **cell type**, and the second the **rotation** of the cell (0, 90, 180 or 270 degrees clockwise).

The following image gives an overview of the eight basic types. These can be rotated in steps of 45° and mirrored along the North-South of East-West axis.

![cell types](https://drive.google.com/uc?export=view&id=164iowmfRQ9O34hquxLhO2xxt49NE473P)


Here's the list of transitions as defined in flatland.core.grid.rail_env_grid.RailEnvTransitions

```
transition_list = [int('0000000000000000', 2),  # empty cell - Case 0
                   int('1000000000100000', 2),  # Case 1 - straight
                   int('1001001000100000', 2),  # Case 2 - simple switch
                   int('1000010000100001', 2),  # Case 3 - diamond drossing
                   int('1001011000100001', 2),  # Case 4 - single slip
                   int('1100110000110011', 2),  # Case 5 - double slip
                   int('0101001000000010', 2),  # Case 6 - symmetrical
                   int('0010000000000000', 2),  # Case 7 - dead end
                   int('0100000000000010', 2),  # Case 1b (8)  - simple turn right
                   int('0001001000000000', 2),  # Case 1c (9)  - simple turn left
                   int('1100000000100010', 2)]  # Case 2b (10) - simple switch mirrored
```

RailEnv accepts a rail_generator parameter, which is used to generate the rail networks. When creating fixed networks, we use the `rail_from_manual_specifications_generator`.

In [ ]:
env = RailEnv(width=rail_shape[1],
              height=rail_shape[0],
              rail_generator=rail_from_manual_specifications_generator(specs),
              number_of_agents=1
              )

A call to `reset()` is necessary to fully initialize the environment. As usual with gym-like environment, this returns the initial observation as a results.

In [ ]:
observation = env.reset()

As we have seen above, the environment provides very complete observations by default. You typically won't use this object as-is. 

One of the main objectives of the Flatland challenge is to find a suitable observation to solve the task at hand. We will see how this works in the next notebook.

Random rail network
---

You will generally want to create randomly-generated environment, instead of specifying each cell by hand. You can use the `random_rail_generator` for this purpose.

It allows you to specify weights for each cell type to increase or decrease their proportion in the generated rail networks.

In [ ]:
# Random railway: built from list of probability per cell type
transition_probability = [1.0,  # empty cell - Case 0
                          1.0,  # Case 1 - straight
                          1.0,  # Case 2 - simple switch
                          0.3,  # Case 3 - diamond drossing
                          0.5,  # Case 4 - single slip
                          0.5,  # Case 5 - double slip
                          0.2,  # Case 6 - symmetrical
                          0.0,  # Case 7 - dead end
                          0.2,  # Case 8 - turn left
                          0.2,  # Case 9 - turn right
                          1.0]  # Case 10 - mirrored switch

random_env = RailEnv(width=15,
              height=15,
              rail_generator=random_rail_generator(
                        cell_type_relative_proportion=transition_probability
                        ),
              number_of_agents=1)

random_env.reset();

Animated rendering
---

TODO move somewhere else

We will use [ipycanvas](https://github.com/martinRenou/ipycanvas) to cleanly display the environment in the notebook.
Note that Flatland comes with multiple rendering options.

In [ ]:
from ipycanvas import Canvas

In [ ]:
from flatland.utils.rendertools import AgentRenderVariant

env_renderer = render_pil = RenderTool(env, gl="PILSVG",
                                       agent_render_variant=AgentRenderVariant.ONE_STEP_BEHIND,
                                       show_debug=False,
                                       screen_height=1000,  # Adjust these parameters to fit your resolution
                                       screen_width=1300)  # Adjust these parameters to fit your resolution

env_renderer.reset()

In [ ]:
render_pil.render_env(show=False, show_observations=False, show_predictions=False, show_agents=True)
img = render_pil.get_image()

canvas = Canvas(size=(img.shape[0], img.shape[1]))
canvas.put_image_data(img)

In [ ]:
canvas